In [1]:
# Load the autoreload extension
%load_ext autoreload
# Reload all modules (except those imported with %aimport) automatically
%autoreload 2

import sys
sys.path.append("../GP_Copilot_Webapp")

import concurrent.futures
import time
from requests.exceptions import Timeout


In [2]:
## get a list of modules that actually exist: 
modules_data = pd.read_csv('/Users/edwinhuang/Downloads/JobsOnCloud_2015-2025 - Sheet1-2.csv', header = 1)
filtered = modules_data[modules_data['jobcount'] > 35]
modules_that_exist = filtered.TASK_NAME.values.tolist()

In [3]:
## delete collections to restart 
## creating different vector stores: 

chroma_path = os.path.join('../GP_Copilot_Webapp', 'chroma')
print(f'chroma path exists? {os.path.exists(chroma_path)}')
client = chromadb.PersistentClient(path = chroma_path)


chroma path exists? True


In [38]:
collections_to_create = ['genepattern_guide', 'genepattern_module_manifests','genepattern_module_readmes', 'genepattern_module_wrappers', 'genepattern_module_documentations', 'genepattern_threads']
for col in collections_to_create:
    client.get_or_create_collection(col)
    # client.delete_collection(col)

In [22]:
def save_text_to_file(text, filename):

    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)

def agent_summarization(agent, documents, document_type):
    """
    Summarizes a documentation via agent
    """


    return result
    for doc in tqdm(documents, desc=f"Summarizing {document_type} with LLM"):
        output = agent(doc.page_content)
        output_fp = f'{doc.metadata["module_name"]}_{document_type}_LLM_summarized.txt'
        doc.page_content = output
        save_text_to_file(output, os.path.join('LLM generated docs', document_type, output_fp))
        

In [24]:
save_text_to_file('test', os.path.join('LLM_generated_docs','manifests', 'manifest_test.txt'))


In [41]:
## paths
manifests = '/Users/edwinhuang/Documents/GitHub/llm-hackathon/scrapes/manifests'
documentations = '/Users/edwinhuang/Documents/GitHub/llm-hackathon/scrapes/doc'
wrappers = '/Users/edwinhuang/Documents/GitHub/llm-hackathon/scrapes/wrappers'
threads = '/Users/edwinhuang/Documents/GitHub/llm-hackathon/scrapes/threads2'
readmes = '/Users/edwinhuang/Desktop/documentations/readmes'

manifest_docs, readme_docs, thread_docs, wrappers, documentation_files, guides = obtain_raw_documents(manifests, documentations, wrappers, threads, readmes, modules_that_exist)

There are a total of 155 manifests.
There are a total of 78 read mes.
There are a total of 902 threads. 
There are a total of 376 wrappers. 
MuPDF error: format error: No default Layer config

MuPDF error: format error: No default Layer config

MuPDF error: format error: No default Layer config

MuPDF error: format error: No default Layer config

MuPDF error: format error: No default Layer config

MuPDF error: format error: No default Layer config

MuPDF error: format error: No default Layer config

MuPDF error: format error: No default Layer config

There are a total of 257 documentation_files. 
There are a total of 280 genepattern guides. 


In [42]:
print("len(wrappers):", len(wrappers))
print("len(thread_docs):", len(thread_docs))
print("len(manifest_docs):", len(manifest_docs))
print("len(readme_docs):", len(readme_docs))
print("len(documentation_files):", len(documentation_files))
print("len(guides):", len(guides))

len(wrappers): 376
len(thread_docs): 902
len(original_questions): 596
len(answers): 596
len(manifest_docs): 155
len(readme_docs): 78
len(documentation_files): 257
len(guides): 280


In [ ]:
# for doc in tqdm(wrappers, desc="Summarizing wrappers with LLM"):
#     output = code_agent(doc.page_content)
#     output_fp = f'{doc.metadata["module_name"]}_wrappers_LLM_summarized.txt'
#     doc.page_content = output
#     save_text_to_file(output, os.path.join('LLM generated docs', output_fp))

In [30]:
# for doc in tqdm(readme_docs, desc="Summarizing readmes with LLM"):
#     output = code_agent(doc.page_content)
#     output_fp = f'{doc.metadata["module_name"]}_readme_LLM_summarized.txt'
#     doc.page_content = output
#     save_text_to_file(output, os.path.join('LLM generated docs', output_fp))

There are a total of 78 read mes.


In [50]:
i = 0
for doc in wrappers+thread_docs + manifest_docs + readme_docs + documentation_files:
    if 'source' in doc.metadata.keys():
        i+=1
i

1768

In [46]:
for doc in tqdm(documents, desc=f"Summarizing {document_type} with LLM"):
    output = agent(doc.page_content)
    output_fp = f'{doc.metadata["module_name"]}_{document_type}_LLM_summarized.txt'
    doc.page_content = output
    save_text_to_file(output, os.path.join('LLM generated docs', document_type, output_fp))

1768

In [47]:
agg = wrappers[:3]+thread_docs[:3] + manifest_docs[:3] + readme_docs[:3] + documentation_files[:3]

In [132]:
# Function to handle parallel execution
def agent_run(agent, document):
    
    try:
        result = agent(document)
        document.page_content = result
    except Exception as e:
        print(f"Error encountered: {e}")
        tqdm.write("Rate limit reached, sleeping for 61 seconds...")
        time.sleep(61)  # Sleep and retry
        result = agent(document)
        document.page_content = result

    
    return document
        
def run_parallel(documents, agent):
    tasks = []
    
    # Calculate total iterations for progress bar
    total_iterations = len(documents)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor, tqdm(total=total_iterations, desc="Running summarization") as pbar:
        future_to_params = {}
        # Submit tasks
        for doc in documents:
            future = executor.submit(agent_run, agent, doc)
            future_to_params[future] = (agent, doc)
        
        # Collect results
        results = []
        for future in concurrent.futures.as_completed(future_to_params):
            try:
                results.append(future.result())  # Get the result
            except Exception as e:
                tqdm.write(f"Unexpected error: {e}")
            pbar.update(1)  # Update progress bar

    return results




In [133]:
['Seurat' in doc.page_content for doc in guides[0:5]]

[False, False, False, False, False]

In [134]:
from copilot.LLM_aided_rag import *
results = run_parallel(guides[60:63], guides_agent)

Running summarization: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.53s/it]


In [136]:
from copilot.LLM_aided_rag import *
llm_wrappers = run_parallel(wrappers, manifest_agent)
llm_manifests = run_parallel(manifest_docs,manifest_agent)
llm_readmes = run_parallel(readme_docs, documentation_agent)
llm_documentations = run_parallel(documentation_files, documentation_agent)
llm_guides = run_parallel(guides, guides_agent)
print("len(llm_wrappers):", len(llm_wrappers))
print("len(llm_manifests):", len(llm_manifests))
print("len(llm_readmes):", len(llm_readmes))
print("len(threads):", len(threads))
print("len(llm_documentations):", len(llm_documentations))
print("len(llm_guides):", len(llm_guides))


Running summarization: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280/280 [04:59<00:00,  1.07s/it]

len(llm_wrappers): 376
len(llm_manifests): 155
len(llm_readmes): 78
len(threads): 65
len(llm_documentations): 257
len(llm_guides): 280


In [142]:
len(thread_docs)

902

In [ ]:
## add metadata to page content:

for sec in [wrappers, thread_docs, manifest_docs, readme_docs, documentation_files, guides]:
    for doc in sec:
        doc.page_content = metadata_formatter(doc.metadata) + doc.page_content




In [ ]:
print(documentation_files[7])

In [141]:
## load into respective chroma dbs

def load_files_into_vector_store(vector_store, documents):
    """
    Loads files into the vector , returns retriever
    """
    uuids = [str(uuid4()) for _ in range(len(documents))]
    # Load the documents into the vector store
    
    # Assuming 'documents' is a list of texts and 'uuids' is a list of corresponding IDs
    for doc, doc_id in tqdm(zip(documents, uuids), total=len(documents), desc="Adding Documents"):
        vector_store.add_documents([doc], ids=[doc_id])

    print(f'Loaded {len(documents)} documents into the vector store')
    return vector_store.as_retriever()

from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

embeddings = get_embeddings('fast')
chroma_path = os.path.join('../GP_Copilot_Webapp', 'chroma')
os.path.exists(chroma_path)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

True

In [146]:
doc_assignments = dict(zip(collections_to_create, [llm_guides, llm_manifests, llm_readmes, llm_wrappers, llm_documentations, thread_docs]))
collections_to_create

['genepattern_guide',
 'genepattern_module_manifests',
 'genepattern_module_readmes',
 'genepattern_module_wrappers',
 'genepattern_module_documentations',
 'genepattern_threads']

In [147]:
doc_assignments = dict(zip(collections_to_create, [llm_guides, llm_manifests, llm_readmes, llm_wrappers, llm_documentations, thread_docs]))
for col in collections_to_create:
    print(f'loading documents for {col}')
    client.get_or_create_collection(col)
    vector_store = Chroma(client=client,
                    embedding_function = embeddings,
                    persist_directory = chroma_path,
                    collection_name=col)
    ret = load_files_into_vector_store(vector_store, doc_assignments[col])

loading documents for genepattern_guide


Adding Documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280/280 [00:21<00:00, 12.95it/s]


Loaded 280 documents into the vector store
loading documents for genepattern_module_manifests


Adding Documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 155/155 [00:12<00:00, 11.93it/s]


Loaded 155 documents into the vector store
loading documents for genepattern_module_readmes


...

Loaded 376 documents into the vector store
loading documents for genepattern_module_documentations


Adding Documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 257/257 [00:21<00:00, 12.18it/s]


Loaded 257 documents into the vector store
loading documents for genepattern_threads


Adding Documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 902/902 [01:23<00:00, 10.84it/s]

Loaded 902 documents into the vector store


In [ ]:
## validate that docs are there:
for col in collections_to_create:
    get_vector_store(chroma_path, col)
    # client.delete_collection(col)

In [ ]:
get_vector_store("GP_Copilot/chroma", 'my_vector_store')

In [ ]:
collections_to_create

In [150]:
for doc in llm_documentations:
    if 'Seurat.QC' in doc.page_content:
        print('-----------------------------------------')
        print(doc)

-----------------------------------------
page_content='### **Section 1: Summary**

The document is about Seurat.Preprocessing, a GenePattern module that implements preprocessing steps for single-cell datasets using the Seurat package (version 4.0.3). Seurat.Preprocessing is part of the broader Seurat pipeline, which is a suite of tools for single-cell analysis. This module is designed to prepare datasets for downstream analyses and visualizations, such as clustering and dimensionality reduction. The intended use case is for researchers working with single-cell RNA sequencing data to preprocess their data for further analysis. The biological context is focused on single-cell datasets, potentially from various species, diseases, or tissues. Key tools and methods mentioned include Seurat, GenePattern, and R.

### **Section 2: Structured Annotations**

- `analysis_type`: Single-cell RNA sequencing data preprocessing
- `use_case`: Preprocessing of single-cell datasets for downstream analys

In [161]:
vector_db_query = f"""
        "Seurat": ["Seurat.Preprocessing", "Seurat.Clustering", "Seurat.VisualizeMarkerExpression", "Seurat.IntegrateData", "Seurat.QC"]
        """
retrieved_docs = retrieve_documents(chroma_path, vector_db_query, ['genepattern_module_documentations'])
    

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Vector store: genepattern_module_documentations found. Number of documents in collection: 257
Using llama-mini


In [164]:
for doc in retrieved_docs:
    if 'Seurat' in doc.page_content:
        print('----------------------------------------------------')
        print(doc)

----------------------------------------------------
page_content='### **Section 1: Summary**

The document is about Seurat.Preprocessing, a GenePattern module that implements preprocessing steps for single-cell datasets using the Seurat package (version 4.0.3). Seurat.Preprocessing is part of the broader Seurat pipeline, which is a suite of tools for single-cell analysis. This module is designed to prepare datasets for downstream analyses and visualizations, such as clustering and dimensionality reduction. The intended use case is for researchers working with single-cell RNA sequencing data to preprocess their data for further analysis. The biological context is focused on single-cell datasets, potentially from various species, diseases, or tissues. Key tools and methods mentioned include Seurat, GenePattern, and R.

### **Section 2: Structured Annotations**

- `analysis_type`: Single-cell RNA sequencing data preprocessing
- `use_case`: Preprocessing of single-cell datasets for downst